In [1]:
from pathlib import Path

from IPython.display import HTML

from gemdat import Jumps, Trajectory, plots

from gemdat.utils import VASPRUN

/home/stef/python/gemdat/.venv/lib64/python3.12/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
# Get our trajectory
trajectory = Trajectory.from_vasprun(VASPRUN)

# Create a new trajectory object that only consists of Lithium
diff_trajectory = trajectory.filter('Li')
diff_trajectory

Full Formula (Li48)
Reduced Formula: Li
abc   :  19.873726   9.919447   9.916454
angles:  90.214114  90.859135  89.950474
pbc   :       True       True       True
Constant lattice (True)
Sites (48)
Time steps (5000)

### Converting a Trajectory to a Volume

To convert a `Trajectory` to a Sites object we use the `gemdat.volume.trajectory_to_volume` function provided by gemdat. This function converts a Trajectory to a Volume, a volume in this context is a discretized version of the volume, in which each point in the trajectory is assigned to a bin, and the number of points in bins is counted.

In [3]:
# Lets create a volume from our trajectory of diffusive elements
from gemdat.volume import trajectory_to_volume

# Resolution is the minimum size of the voxels in Angstrom
diff_volume = trajectory_to_volume(diff_trajectory, resolution=0.3)

### Converting a Volume to Sites

We can directly convert a volume to a structure with the  `Volume.to_structure` method. This uses a [watershed](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_watershed.html) algorithm to find the sites.

various helper functions are available to the Volume object that can be used for various purposes [gemdat.Volume API](https://gemdat.readthedocs.io/en/latest/api/gemdat_volume/), for example finding peaks or converting a Volume to a vasp file

In [8]:
# Now convert it to a structure
structure = diff_volume.to_structure(specie='Li', background_level=0.1)
structure[0:10]

[PeriodicSite: Li (7.214, 5.229, 2.009) [0.3636, 0.5273, 0.2091],
 PeriodicSite: Li (8.248, 5.08, 3.038) [0.4161, 0.5123, 0.3137],
 PeriodicSite: Li (3.125, 8.8, 5.397) [0.1591, 0.8879, 0.5485],
 PeriodicSite: Li (4.325, 8.024, 5.877) [0.2197, 0.8098, 0.5976],
 PeriodicSite: Li (3.117, 9.76, 6.566) [0.1591, 0.9848, 0.6667],
 PeriodicSite: Li (1.977, 9.701, 7.965) [0.1023, 0.9792, 0.8068],
 PeriodicSite: Li (16.76, 2.665, 9.515) [0.847, 0.2697, 0.9727],
 PeriodicSite: Li (17.97, 2.203, 9.503) [0.9078, 0.2231, 0.9723],
 PeriodicSite: Li (11.9, 9.459, 8.221) [0.6019, 0.9545, 0.8401],
 PeriodicSite: Li (12.21, 8.683, 9.274) [0.6178, 0.8765, 0.9463]]

In [5]:
# Write it to a vasp file for inspection in external tools
diff_volume.to_vasp_volume(structure, filename='/tmp/example.vasp')

In [9]:
# These sites overlap with neighbours, so remove them
del structure.sites[25]
del structure.sites[26] 

# Generate transitions using newly generated structure
transitions = trajectory.transitions_between_sites(sites=structure, floating_specie='Li')

In [10]:
jumps = Jumps(ransitions, sites=sites, minimal_residence=0)

NameError: name 'sites' is not defined

In [ ]:
fig = plots.jumps_vs_time(jumps=jumps, n_parts=5)
HTML(fig.to_html())

In [ ]:
_ = plots.jumps_3d(jumps=jumps)